<a href="https://colab.research.google.com/github/DuplamenteH/Machine-Learning/blob/master/Clustering_Basico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # *Estudo de alguns algoritmos de clustering ex: K-means, DBSCAN, mean shift*

# Aula 1

### instalação

In [ ]:
pip install plotly==4.1.1


In [ ]:
pip install biokit

### Analise dos dados e importações de libs.

importações

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from biokit.viz import corrplot
import matplotlib.pyplot as plt



In [ ]:
path_arq = 'https://gist.githubusercontent.com/tijptjik/9408623/raw/b237fa5848349a14a14e5d4107dc7897c21951f5/wine.csv'

In [ ]:
df = pd.read_csv(path_arq, sep=',')

In [ ]:
print("Dimensões de Dataset {}, {}".format(df.shape[0], df.shape[1]))
df.head(10)

renomeando as features

In [ ]:
colunas = {
    'Alcohol':'Alcool',
    'Ash' : 'PO',
    'Acl':'Alcalinidade_po',
    'Mg' : 'Magnesio',
    'Color.int' : 'Intensidade_cor'
}
df = df.rename(columns=colunas)
df.head()

In [ ]:
df.describe()

### Insights:

* Vemos que a base de dados não possui dados faltantes.
* Média(e STD) bem variada ao longo do DF.-> Processo de adquação
* Variação bem alta do proline.

### vendo as relações

In [ ]:
matriz_corr = df.corr()

In [ ]:
corr_graf = corrplot.Corrplot(matriz_corr)
corr_graf.plot(upper = 'ellipse', fontsize = 'x-large')
fig = plt.gcf()
fig.set_size_inches(14,14)
fig.show()

### Normalização 

In [ ]:
atributos = df.columns

In [ ]:
atributos

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
min_maxScaler = MinMaxScaler()
np_df = min_maxScaler.fit_transform(df)

In [ ]:
df = pd.DataFrame(np_df, columns=atributos)

In [ ]:
df.describe()

# Aula 02 Kmeans

### K-means

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
group1 = KMeans(n_clusters=4)

In [ ]:
group1.fit(df)

In [ ]:
grupos = group1.labels_

In [ ]:
grupos

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= df['Intensidade_cor'], y = df['Alcool'],
                         mode ='markers',marker=dict(color = group1.labels_.astype(np.float)),
                         text = grupos
                        ))
fig.show()

### testando com outro K




In [ ]:
group2 = KMeans(n_clusters=3)

In [ ]:
group2.fit(df)

In [ ]:
grupos2 = group2.labels_
grupos2

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= df['Intensidade_cor'], y = df['Alcool'],
                         mode ='markers',marker=dict(color = group2.labels_.astype(np.float)),
                         text = grupos2
                        ))
fig.show()

### 3 Dimensões

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x= df['Intensidade_cor'], y = df['Alcool'],z = df['Proline'],
                         mode ='markers',marker=dict(color = group2.labels_.astype(np.float)),
                         text = grupos2
                        ))
fig.show()

In [ ]:
fig.update_layout(scene = dict(
    xaxis_title = 'Intensidade de Cor',
    yaxis_title = 'Álcool',
    zaxis_title = 'Proline'
))

fig.show()

### Criando centros

In [ ]:
centros = pd.DataFrame(group2.cluster_centers_)

In [ ]:
centros.columns = atributos

In [ ]:
centros.head()

In [ ]:
fig.add_trace(go.Scatter3d(
    x= centros['Intensidade_cor'],
    y= centros['Alcool'],
    z= centros['Proline'],
    mode = 'markers',
    marker = dict(color = 'red'),
    text = [0,1,2]
))
fig.show()

### Testando a previsão

In [ ]:
nova_amostra = [10,10,1.5,27,4,14,38,0.7,10, 1, 10,4 ,16,12]
label_nova_amostra = group2.predict([nova_amostra])
print(label_nova_amostra)

# Aula 03 DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
dbscan = DBSCAN(eps = 1.31, min_samples=15, metric='manhattan')
dbscan.fit(df)

In [ ]:
dbscan.labels_

In [ ]:
np.unique(dbscan.labels_)

In [ ]:
dbscan2 = DBSCAN(eps=0.5, min_samples=15, metric='euclidean')
dbscan2.fit(df)
dbscan2.labels_

In [ ]:
np.unique(dbscan2.labels_)

In [ ]:
labels = dbscan2.labels_+1
print(labels)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=df['Intensidade_cor'],
                         y=df['Alcool'], z = df['Proline'],
                         mode='markers',                      
                      marker=dict(color =labels.astype(np.float)),
                        text=labels))
fig.show()

# Aula 04 mean shift

In [ ]:
from sklearn.cluster import MeanShift, estimate_bandwidth

In [ ]:
meanShift = MeanShift()

In [ ]:
meanShift.fit(df)

In [ ]:
meanShift.labels_

In [ ]:
BW = estimate_bandwidth(df,quantile=0.3)
agrupador = MeanShift(bandwidth=BW)
agrupador.fit(df)
agrupador.labels_

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=df['Intensidade_cor'],
                         y=df['Alcool'], z=df['Proline'],
                         mode = 'markers',                      
                          marker = dict(color = agrupador.labels_.astype(np.float)),
                        text = [0, 1, 2]))
fig.show()

In [ ]:
fig.update_layout(scene = dict(
                    xaxis_title='Intensidade de Cor',
                    yaxis_title='Àlcool',
                    zaxis_title='Proline'))
fig.show()

In [ ]:
centros = pd.DataFrame(agrupador.cluster_centers_)
centros.columns = atributos

In [ ]:
fig.add_trace(go.Scatter3d(x=centros['Intensidade_cor'],
                         y=centros['Alcool'], z=centros['Proline'],
                         mode = 'markers',                  
                          marker = dict(color = 'red'),
                        text = [0, 1, 2]))
fig.show()

# Aula 05 Coeficiente de Silhueta

In [ ]:
agrupador = KMeans(n_clusters=5)
agrupador.fit(df)
agrupador.labels_

In [ ]:
labels = agrupador.labels_

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=df['Intensidade_cor'],
                         y=df['Alcool'], z=df['Proline'],
                         mode = 'markers',                      
                          marker = dict(color = agrupador.labels_.astype(np.float)),
                        text =labels ))
fig.show()

## Coeficiente de Silhueta K-means

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
faixa_n_cluster = [i for i in range(2,10)]
print(faixa_n_cluster)

In [ ]:
from sklearn.cluster import KMeans
valores_silhueta=[]

In [ ]:
for k in faixa_n_cluster:
    agrupador = KMeans(n_clusters=k)
    labels = agrupador.fit_predict(df)
    media_silhueta = silhouette_score(df,labels)
    valores_silhueta.append(media_silhueta)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = faixa_n_cluster, y=valores_silhueta))
fig.update_layout(
    title = "Valores de Silhueta Médios",
    xaxis_title = "Número de cluster",
    yaxis_title = "Valor médio de silhueta"
)
fig.show()


## Coeficiente de Silhueta Mean-Shift




In [ ]:
faixa_quantil = np.linspace(0.02, 0.5, 30)
faixa_quantil

In [ ]:
valores_silhueta_mean_shift=[]
quantil_plot = []
for quantil in faixa_quantil:
    bw = estimate_bandwidth(df, quantile = quantil)
    agrupador = MeanShift(bandwidth=bw)
    labels = agrupador.fit_predict(df)
    if (len(np.unique(labels))< len(df) and len(np.unique(labels))>1): 
        media_silhueta = silhouette_score(df,labels)
        valores_silhueta_mean_shift.append(media_silhueta)
        quantil_plot.append(quantil)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = quantil_plot,
                         y= valores_silhueta_mean_shift))
fig.update_layout(dict(xaxis_title='Quantil'),
                        yaxis_title='Media silhueta')
fig.show()